In [1]:
%matplotlib inline
# disable autoreload for global dataset
%load_ext autoreload
%autoreload 2

In [2]:
from learn import *

In [3]:
num_gpus = 2
gpu_start = 8
num_workers = 3 * num_gpus
device_ids = range(gpu_start, gpu_start + num_gpus)
torch.cuda.set_device(gpu_start)

bs = 20

## Plot predictions

In [4]:
model_name = 'deeplab'

In [5]:
learn, denorm, data = learner_on_dataset(datapaths[2], bs, device_ids, num_workers, model_name=model_name,
                                         debug=False, global_dataset=False, num_slice=9, sz=256)
(trn_x,trn_y), (val_x,val_y) = data

HBox(children=(IntProgress(value=0, description='ims', max=3207), HTML(value='')))

HBox(children=(IntProgress(value=0, description='ims', max=1375), HTML(value='')))


Data finished loading: data/train/AOI_4_Shanghai_Train


In [13]:
#resume
learn, denorm, data = learner_on_dataset(datapaths[2], bs, device_ids, num_workers, model_name=model_name,
                                         debug=False, global_dataset=False, data=data, num_slice=9, sz=256)
(trn_x,trn_y), (val_x,val_y) = data

Data finished loading: data/train/AOI_4_Shanghai_Train


## plots

In [6]:
learn.load('deeplab-shanghai')

In [7]:
%%time
preds = learn.predict()

CPU times: user 2h 19min 2s, sys: 12min 49s, total: 2h 31min 51s
Wall time: 4min 51s


In [ ]:
print('hi')

In [ ]:
preds.shape

## CRF

In [ ]:
import numpy as np
try:
    import cPickle as pickle
except ImportError:
    import pickle

from pystruct import learners
import pystruct.models as crfs
from pystruct.utils import SaveLogger


data_train = pickle.load(open("data_train_dict.pickle"))
C = 0.01

n_states = 21
print("number of samples: %s" % len(data_train['X']))
class_weights = 1. / np.bincount(np.hstack(data_train['Y']))
class_weights *= 21. / np.sum(class_weights)
print(class_weights)

model = crfs.EdgeFeatureGraphCRF(inference_method='qpbo',
                                 class_weight=class_weights,
                                 symmetric_edge_features=[0, 1],
                                 antisymmetric_edge_features=[2])

experiment_name = "edge_features_one_slack_trainval_%f" % C

ssvm = learners.NSlackSSVM(
    model, verbose=2, C=C, max_iter=100000, n_jobs=-1,
    tol=0.0001, show_loss_every=5,
    logger=SaveLogger(experiment_name + ".pickle", save_every=100),
    inactive_threshold=1e-3, inactive_window=10, batch_size=100)
ssvm.fit(data_train['X'], data_train['Y'])

data_val = pickle.load(open("data_val_dict.pickle"))
y_pred = ssvm.predict(data_val['X'])

# we throw away void superpixels and flatten everything
y_pred, y_true = np.hstack(y_pred), np.hstack(data_val['Y'])
y_pred = y_pred[y_true != 255]
y_true = y_true[y_true != 255]

print("Score on validation set: %f" % np.mean(y_true == y_pred))